Activate the environment that has been [instantiated](README.md).

In [17]:
using Pkg
pkg"activate ."
pkg"instantiate"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %

Load StructJuMP

In [2]:
using StructJuMP

We now create a structured model `m`. At this point the number of second-stage submodels is set, referring to them as *scenarios*.

In [3]:
num_scenarios=2
m = StructuredModel(num_scenarios=num_scenarios)

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

The model is printed and we see it is empty. Like the underlying modeling framework [JuMP](https://github.com/JuliaOpt/JuMP.jl), there are 3 basic components of a model object: `@variable`, `@constraint`, and `@objective`. In addition, nonlinear constraints and objectives are set via `@NLconstraint` and `@NLobjective`.

In [4]:
n=3
initvec=zeros(n)
@variable(m, x[i=1:n] >= 0, start=initvec[i])
@variable(m, y[i=1:n] >= 0, start=initvec[i])
for i in 1:n
    @NLconstraint(m, x[i] * y[i] >= i)
end
@NLobjective(m, :Min, sum(x[i]*y[i] for i=1:n))
m

Minimization problem with:
 * 0 linear constraints
 * 3 nonlinear constraints
 * 6 variables
Solver is default solver

The above defines the first stage described by `m`. Notice how we can print the model. We now want to create the 2 second-stage models and connect them to the first stage `m`. We create the scenarios using a `for` loop. `StructuredModel` creates a submodel `mk` attached to the parent `m` and with a unique identifier `i`.

In [5]:
for k in 1:num_scenarios
    mk = StructuredModel(parent=m,id=k)
    @variable(mk, w[i=1:n] >= 0, start=initvec[i])
    @constraint(mk, sum(i*w[i] for i=1:n) == k)
    @NLconstraint(mk, sum(x[i] - w[i] for i=1:n) <= 0)
    @NLobjective(mk, :Min, sum(w[i]*w[i] for i=1:n))
end

Let's print again the entire model.

In [6]:
print(m)

Min x[1] * y[1] + x[2] * y[2] + x[3] * y[3]
Subject to
 x[1] * y[1] - 1.0 ≥ 0
 x[2] * y[2] - 2.0 ≥ 0
 x[3] * y[3] - 3.0 ≥ 0
 x[i] ≥ 0 ∀ i ∈ {1,2,3}
 y[i] ≥ 0 ∀ i ∈ {1,2,3}
*** children ***
Child ID 2:
Min w[1] * w[1] + w[2] * w[2] + w[3] * w[3]
Subject to
 w[1] + 2 w[2] + 3 w[3] = 2
 ((x[1] - w[1]) + (x[2] - w[2]) + (x[3] - w[3])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,3}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
*** children ***

Child ID 1:
Min w[1] * w[1] + w[2] * w[2] + w[3] * w[3]
Subject to
 w[1] + 2 w[2] + 3 w[3] = 1
 ((x[1] - w[1]) + (x[2] - w[2]) + (x[3] - w[3])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,3}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
*** children ***



We have now created a model. In order to solve it we need to load a solver interface. `StructJuMP` was initially designed as a generic package to run with a variety of solvers. Currently, we support `Ipopt` and the large-scale interior point solver [PIPS](https://github.com/Argonne-National-Laboratory/PIPS). These are loaded via the package `StructJuMPSolverInterface`.

In [7]:
using StructJuMPSolverInterface

In a distributed parallel setting using `MPI`, we can print the local children.

In [8]:
getLocalChildrenIds(m)

2-element Array{Int64,1}:
 1
 2

Here we see both children as we run sequentially. We can access and print the local children at any time.

In [9]:
for i in getLocalChildrenIds(m)
    print(getchildren(m)[i])
end

Min w[1] * w[1] + w[2] * w[2] + w[3] * w[3]
Subject to
 w[1] + 2 w[2] + 3 w[3] = 1
 ((x[1] - w[1]) + (x[2] - w[2]) + (x[3] - w[3])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,3}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
*** children ***
Min w[1] * w[1] + w[2] * w[2] + w[3] * w[3]
Subject to
 w[1] + 2 w[2] + 3 w[3] = 2
 ((x[1] - w[1]) + (x[2] - w[2]) + (x[3] - w[3])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,3}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
*** children ***


In [10]:
using Ipopt

In [11]:
solve(m;solver="Ipopt")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:       18
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

:Solve_Succeeded

Ipopt converges in around 80 iterations and returns with the solutions stored in the variables. We can access them using the variable name as an identifier.

In [12]:
# First stage

println("First stage")
println("x[]= ", getvalue(m[:x]))
println("y[]= ", getvalue(m[:y]))

println("Second stage")
# All the children
for k in 1:num_scenarios
   println("Child ", k, ": w[]= ", getvalue(getchildren(m)[k][:w]))
end

First stage
x[]= [0.00167798, 0.00237362, 0.00290732]
y[]= [595.953, 842.594, 1031.88]
Second stage
Child 1: w[]= [0.0714286, 0.142857, 0.214286]
Child 2: w[]= [0.142857, 0.285714, 0.428571]
